In [27]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from rl.agents import 

class TSPAgent(Agent):
    def __init__(self, model, nb_actions, memory, policy=None, test_policy=None, enable_double_dqn=True, enable_dueling_network=False, dueling_type='avg', *args, **kwargs):
        super(TSPAgent, self).__init__(*args, **kwargs)
        self.model = model
        self.nb_actions = nb_actions
        self.memory = memory
        self.policy = policy
        self.test_policy = test_policy
        self.enable_double_dqn = enable_double_dqn
        self.enable_dueling_network = enable_dueling_network
        self.dueling_type = dueling_type

    def forward(self, observation):
        # predict the action distribution from the model
        q_values = self.model.predict(observation)
        action = np.argmax(q_values[0])  # choose the action with the highest Q-value
        return action, q_values

ImportError: cannot import name 'Agent' from 'rl.agents' (c:\Users\fabian.woellenweber\AppData\Local\Programs\Python\Python310\lib\site-packages\rl\agents\__init__.py)

In [18]:
import gym
import numpy as np

class TSPEnv(gym.Env):
    def __init__(self, cities, distances):
        self.cities = cities
        self.distances = distances
        self.num_cities = len(cities)
        self.action_space = gym.spaces.Discrete(self.num_cities)
        self.observation_space = gym.spaces.Box(0, self.num_cities, shape=(self.num_cities,))
        print(self.observation_space)
    
    def reset(self):
        self.state = np.zeros(self.num_cities)
        print(self.state)
        self.state[0] = 1
        self.visited = [0]
        return self.state
    
    def step(self, action):
        self.state[action] = 1
        if(action not in self.visited):
            self.visited.append(action)
        reward = -self.distances[self.visited[-2], action]
        done = len(self.visited) == self.num_cities
        return self.state, reward, done, {}



# print(state)  # should print [1, 1, 0, 0, 0]
# print(reward) # should print -2448
# print(done)   # should print False


In [23]:
cities = ['New York', 'Chicago', 'Denver', 'San Francisco', 'Los Angeles']
distances = np.array([[0, 2448, 1434, 1260, 2045],
                      [2448, 0, 2546, 959, 2367],
                      [1434, 2546, 0, 2408, 1745],
                      [1260, 959, 2408, 0, 2295],
                      [2045, 2367, 1745, 2295, 0]])

                      
env = TSPEnv(cities, distances)
env.reset()



Box(0.0, 5.0, (5,), float32)
[0. 0. 0. 0. 0.]


array([1., 0., 0., 0., 0.])

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent, DDPGAgent, ContinuousDQNAgent
from rl.policy import EpsGreedyQPolicy, BoltzmannQPolicy
from rl.memory import SequentialMemory
from tensorflow.keras.layers import Lambda


# Learning factors...
EPISODES = 50
discount_factor = 0.99
learning_rate = 0.001
epsilon_start = 1.0
epsilon_decay_steps = 0.999
epsilon_end = 0.01
batch_size = 64
train_start = 1000

# create replay memory using deque
#memory = deque(maxlen=2000)

# Create gym environment
# Create the TSP environment with a set of cities and distances between them
cities = ['New York', 'Chicago', 'Denver', 'San Francisco', 'Los Angeles']
distances = np.array([[0, 2448, 1434, 1260, 2045],
                      [2448, 0, 2546, 959, 2367],
                      [1434, 2546, 0, 2408, 1745],
                      [1260, 959, 2408, 0, 2295],
                      [2045, 2367, 1745, 2295, 0]])

env = TSPEnv(cities, distances)

# Reset the environment and take an action
state = env.reset()
state, reward, done, _ = env.step(2)

state_size = env.observation_space
action_size = env.action_space

# Define the optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.mean_squared_error

# Create a replay buffer to store transitions
replay_buffer = []

# Define the update frequency and batch size
update_frequency = 10
batch_size = 10

# The following function creates a neural network which is used as an 
# approximate Q function
# Input: state 
# Output: Q Value of each action
def build_model(state_size, action_size):
    model = tf.keras.Sequential()
    model.add(Dense(256, input_dim=state_size, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(action_size, activation='linear'))
    model.summary()
    return model

# Create the TSP agent
def build_agent(model, action_size):
    # Use Epsilon-Greedy policy for exploration
    policy = BoltzmannQPolicy()
    # Create memory for storing transitions
    memory = SequentialMemory(limit=50000, window_length=1)
    # Create the DQN agent
    agent = DQNAgent(model, memory=memory, policy=policy, nb_actions=action_size, nb_steps_warmup=10, target_model_update=1e-2)
    
    return agent



# Create the TSP model
model = build_model(state_size, action_size)
model.output
# Create the TSP agent
agent = build_agent(model, action_size)
# Train the agent
agent.compile(Adam(lr=1e-3), metrics=['mae'])
agent.fit(env, nb_steps=5000, visualize=False, verbose=1)


TypeError: Error converting shape to a TensorShape: Dimension value must be integer or None or have an __index__ method, got value 'Box(0.0, 5.0, (5,), float32)' with type '<class 'gym.spaces.box.Box'>'.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential

## Initialize important variables
env_name = 'TSPEnv'
bRender = False
load_model = False

# Learning factors...
EPISODES = 99
discount_factor = 0.99
learning_rate = 0.001
epsilon_start = 1.0
epsilon_decay_steps = 0.999
epsilon_end = 0.01
batch_size = 64
train_start = 1000

# create replay memory using deque
#memory = deque(maxlen=2000)

# Create gym environment
env = TSPEnvironment(EPISODES)

state_size = env.observation_space.n
action_size = env.action_space.n

# Define the optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.mean_squared_error

# Create a replay buffer to store transitions
replay_buffer = []

# Define the update frequency and batch size
update_frequency = 10
batch_size = 10

# The following function creates a neural network which is used as an 
# approximate Q function
# Input: state 
# Output: Q Value of each action
def build_model():
    model = Sequential()
    model.add(Dense(32, input_dim=state_size, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(action_size, activation='linear', kernel_initializer='he_uniform'))
    model.summary()
    model.compile(loss='mse', optimizer=Adam(learning_rate=learning_rate))
    return model

# after some time interval update the target model to be same with model
# This is done using the following function
def update_target_model():
    target_model.set_weights(model.get_weights())

# create main model and target model
model = build_model()
target_model = build_model()

# We call that function in the beginning to make sure model and target_model have the same weights initally
update_target_model()

# Run the TSP loop
num_episodes = 50
for episode in range(num_episodes):
    # Initialize the environment and state
    state = env.reset()

    # Calculate the epsilon for this episode
    epsilon = epsilon_end + (epsilon_start - epsilon_end) * np.exp(-episode / epsilon_decay_steps)

    while True:
        # Select an action using the DQN model and an epsilon-greedy policy
        if np.random.rand() < epsilon:
            # Explore: choose a random action
            action = np.random.randint(env.action_space.n)
        else:
            # Exploit: choose the action with the highest Q-value
            action = np.argmax(model(np.expand_dims(state, axis=0)).numpy())

        # Take the action and observe the reward and next state
        next_state, reward, done, _ = env.step(action)

        # Store the transition in the replay buffer
        replay_buffer.append((state, action, reward, next_state))

        # If the replay buffer has reached a certain size, sample a batch of transitions and update the model
        if len(replay_buffer) >= batch_size:
            # Sample a batch of transitions
            transitions = np.random.sample(replay_buffer,batch_size)
            states, actions, rewards, next_states = zip(*transitions)

            # Convert the states and next_states to tensors
            states = tf.convert_to_tensor(states, dtype=tf.float32)
            next_states = tf.convert_to_tensor(next_states, dtype=tf.float32)
            
            print(f'States: {states}')
            print(f'States: {next_states}')

            # Compute the Q-values for the current states and the next states
            current_q_values = model(states)
            next_q_values = target_model(next_states)

            # Compute the expected Q-values
            expected_q_values = rewards + np.max(next_q_values.numpy(), axis=1)

            # Compute the loss
            loss = loss_fn(expected_q_values, current_q_values)

            # Update the model
            optimizer.minimize(loss, model.trainable_variables)

            # Clear the replay buffer
            replay_buffer = []

        # Update the target model every `update_frequency` steps
        if episode % update_frequency == 0:
            update_target_model()

        # Update the state
        state = next_state

        # If the episode is done, break out of the inner loop
        if done:
            break



    
